<a href="https://colab.research.google.com/github/Yashchauhan23/Deep_Learning/blob/main/OptimizerCNNmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Create CNN Model and Optimize it using Keras Tuner**

In [1]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 35.5 MB/s 


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
print(tf.__version__)

2.8.2


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [6]:
train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images = train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  """Entry point for launching an IPython kernel.


In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 28s]
val_accuracy: 0.8644999861717224

Best val_accuracy So Far: 0.9073333144187927
Total elapsed time: 00h 03m 01s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 48)        480       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 48)        20784     
                                                                 
 flatten (Flatten)           (None, 27648)             0         
                                                                 
 dense (Dense)               (None, 32)                884768    
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                                 
Total params: 906,362
Trainable params: 906,362
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.fit(train_images,train_labels,epochs=10,validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1652 - accuracy: 0.9394 - val_loss: 0.2631 - val_accuracy: 0.9105
Epoch 5/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.1272 - accuracy: 0.9531 - val_loss: 0.2821 - val_accuracy: 0.9125
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0979 - accuracy: 0.9636 - val_loss: 0.2949 - val_accuracy: 0.9117
Epoch 7/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0745 - accuracy: 0.9727 - val_loss: 0.3784 - val_accuracy: 0.9008
Epoch 8/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0589 - accuracy: 0.9782 - val_loss: 0.4128 - val_accuracy: 0.9060
Epoch 9/10
1688/1688 [==============================] - 6s 4ms/step - loss: 0.0449 - accuracy: 0.9841 - val_loss: 0.4370 - val_accuracy: 0.9070
Epoch 10/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0368 - accuracy: 0.9865 - val_loss: 0.4871 - val_accuracy